## Applying XGBoost on the outputs of the LSTM Model

In [0]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

--2018-12-04 07:21:50--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2018-12-04 07:21:50--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  82.0MB/s    in 11s     

2018-12-04 07:22:01 (76.6 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [1]:
import pandas as pd
import numpy as np
from keras.preprocessing.text import one_hot, Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model, load_model
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [0]:
np.random.seed(42)

In [0]:
df_train = pd.read_csv('train_data.csv')
df_labels = pd.read_csv('train_labels.csv')

In [0]:
df_train = df_train.astype({'question1': str, 'question2': str})

In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 323144 entries, 0 to 323143
Data columns (total 3 columns):
id           323144 non-null int64
question1    323144 non-null object
question2    323144 non-null object
dtypes: int64(1), object(2)
memory usage: 7.4+ MB


## Prepare the Train Data

## Load Model

In [0]:
model = load_model('bilstm_dbd_dbd_dbd_256.h5')

## Convert model output to XGBoost Features

In [0]:
tokenizer = Tokenizer()
questions = pd.concat([df_train['question1'], df_train['question2']])
tokenizer.fit_on_texts(questions)

vocab_size = len(tokenizer.word_index) + 1

# Integer Encode
q1_int_sequence = tokenizer.texts_to_sequences(df_train['question1'])
q2_int_sequence = tokenizer.texts_to_sequences(df_train['question2'])

q1_padded = pad_sequences(q1_int_sequence, maxlen=25)
q2_padded = pad_sequences(q2_int_sequence, maxlen=25)

In [0]:
X = np.stack((q1_padded, q2_padded), axis=1)
y = df_labels['is_duplicate']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [0]:
Q1_test = X_test[:,0]
Q2_test = X_test[:,1]

In [10]:
output = model.predict([Q1_test, Q2_test], y_test)

TypeError: ignored

## Prepare Submission

In [0]:
submission['test_id'] = df_test['test_id']
submission['is_duplicate'] = predicted

In [0]:
submission.loc[submission.is_duplicate >= 0.5, 'is_duplicate'] = 1
submission['is_duplicate'] = submission['is_duplicate'].astype(int)

In [0]:
submission.head()

In [0]:
submission.to_csv('lstm_bauman_clasweights_1024.csv', index=False)

In [0]:
model.evaluate([Q1_test, Q2_test], y_test)

32315/32315 [==============================] - 107s 3ms/step


[0.36759693546596184, 0.8350920625022925]

In [0]:
model.save('lstm_bauman_more_dense_1024.h5')